# Using Applications

To run different protein design applications, we first have to import everything we need. In this case, we want to test the application ligandmpnn.

In [1]:
import sys
sys.path.append("..") # add the path to the protflow directory (e.g. /home/username/ProtSlurm)
from protflow.poses import Poses
from protflow.tools.ligandmpnn import LigandMPNN
from protflow.jobstarters import SbatchArrayJobstarter, LocalJobStarter

First, we define our jobstarters. We want to test if we can run LigandMPNN on GPU or cpu using the SLURM workload manager as well as run it locally. If SLURM is not installed on your machine, you can skip the parts mentioning it.

In [2]:
slurm_gpu_jobstarter = SbatchArrayJobstarter(max_cores=10, gpus=1)
slurm_cpu_jobstarter = SbatchArrayJobstarter(max_cores=10, gpus=False)

In [3]:
local_jobstarter = LocalJobStarter(max_cores=1)

Next, we have to load our poses. We set the local_jobstarter as default jobstarter.

In [4]:
my_poses = Poses(poses='data/input_pdbs/', glob_suffix='*pdb', work_dir='runners_example', storage_format='csv', jobstarter=local_jobstarter)

To run ligandmpnn, we have to create a runner. Make sure the path to the LigandMPNN script and python path are set in protflow/config.py! The lines there should look like:

#ligandmpnn.py

LIGANDMPNN_SCRIPT_PATH = "/home/user/LigandMPNN/run.py"

LIGANDMPNN_PYTHON_PATH = "/home/user/anaconda3/envs/ligandmpnn_env/bin/python3"

You can set it also when creating the runner, but it is recommended to set it in the config if you want to run it again.

If running this notebook on a cluster, be sure that is also opened from there (e.g. with VS code installed on the cluster)! Otherwise, the /home/ directories won't match since it will look for the files on your local machine and not on the cluster!

In [5]:
ligandmpnn = LigandMPNN()

To run ligandmpnn on our poses, we have to use the .run() function. All tools and metrics should have this function. It is mandatory to provide a unique prefix for each run. Each score generated will be saved to the poses dataframe in the format prefix_scorename. The output files can be found in a folder called prefix in the working_directory set for the input poses. The .run() function always returns poses.

In [6]:
my_poses = ligandmpnn.run(poses=my_poses, prefix='ligmpnn_local', nseq=2, model_type='protein_mpnn')
display(my_poses.df)

,input_poses,poses,poses_description,ligmpnn_local_mpnn_origin,ligmpnn_local_seed,ligmpnn_local_description,ligmpnn_local_sequence,ligmpnn_local_T,ligmpnn_local_id,ligmpnn_local_seq_rec,ligmpnn_local_overall_confidence,ligmpnn_local_ligand_confidence,ligmpnn_local_location
0,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0001,structure_1,94702.0,structure_1_0001,ALAEALRIIVRGIAFMKRYGEKFMKELLAATRALLARIAKDPVLLA...,0.1,1.0,0.540,0.4123,0.4123,/home/tripp/ProtFlow/examples/runners_example/...
1,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0002,structure_1,94702.0,structure_1_0002,PLARALAIIARGLALMAAHGREFWEKLVAATDALIARIAKDPVLLA...,0.1,2.0,0.515,0.4166,0.4166,/home/tripp/ProtFlow/examples/runners_example/...
2,data/input_pdbs/structure_2.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_2_0001,structure_2,94702.0,structure_2_0001,MSPEELEAARREAEYAARLAEARAWLRRELPLFAKELLTAAGKEEL...,0.1,1.0,0.450,0.4009,0.4009,/home/tripp/ProtFlow/examples/runners_example/...
3,data/input_pdbs/structure_2.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_2_0002,structure_2,94702.0,structure_2_0002,MTPEELAAAAAAAARAARLAEGRALLRRELPRFAARLLRAAGREEL...,0.1,2.0,0.425,0.4078,0.4078,/home/tripp/ProtFlow/examples/runners_example/...
4,data/input_pdbs/structure_3.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_3_0001,structure_3,94702.0,structure_3_0001,AAALRALLVALARLAGDPDPAAQDAFLALAIALLAALYLAALDPAL...,0.1,1.0,0.515,0.4612,0.4612,/home/tripp/ProtFlow/examples/runners_example/...
5,data/input_pdbs/structure_3.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_3_0002,structure_3,94702.0,structure_3_0002,MEELEALLVELFRLAGDPDPAAQARYLGLAIALLGALALAALDPAL...,0.1,2.0,0.625,0.4434,0.4434,/home/tripp/ProtFlow/examples/runners_example/...


Notice how the poses dataframe has changed! It now contains all the poses generated from LigandMPNN and the corresponding scores. 

Since we did not provide a jobstarter when we set up ligandmpnn, it ran on the local machine, because it defaulted to the jobstarter we set when creating our poses. We can run LigandMPNN using another jobstarter, either by providing LigandMPNN a default jobstarter or calling the .run function with the jobstarter option.

Since we want to run LigandMPNN on the original poses again, we have to reset to the original input (this is in general not recommended, as it will delete the dataframe and might lead to reuse of prefix, reading in previously generated data).

In [7]:
my_poses.reset_poses(force_reset_df=True)
my_poses = ligandmpnn.run(poses=my_poses, prefix='ligmpnn_cpu', nseq=2, model_type='protein_mpnn', jobstarter=slurm_cpu_jobstarter)
display(my_poses.df)

sbatch: defined options
sbatch: -------------------- --------------------
sbatch: array               : 1-3%10
sbatch: error               : /home/tripp/ProtFlow/examples/runners_example/ligmpnn_cpu//ligandmpnn_217198_slurm.err
sbatch: job-name            : ligandmpnn_217198
sbatch: output              : /home/tripp/ProtFlow/examples/runners_example/ligmpnn_cpu//ligandmpnn_217198_slurm.out
sbatch: verbose             : 3
sbatch: wrap                : eval `sed -n ${SLURM_ARRAY_TASK_ID}p /home/tripp/ProtFlow/examples/runners_example/ligmpnn_cpu//ligandmpnn_217198_cmds`
sbatch: -------------------- --------------------
sbatch: end of defined options
sbatch: debug:  propagating RLIMIT_CPU=18446744073709551615
sbatch: debug:  propagating RLIMIT_FSIZE=18446744073709551615
sbatch: debug:  propagating RLIMIT_DATA=18446744073709551615
sbatch: debug:  propagating RLIMIT_STACK=67108864
sbatch: debug:  propagating RLIMIT_CORE=0
sbatch: debug:  propagating RLIMIT_RSS=18446744073709551615
sbatch: d

,input_poses,poses,poses_description,ligmpnn_cpu_mpnn_origin,ligmpnn_cpu_seed,ligmpnn_cpu_description,ligmpnn_cpu_sequence,ligmpnn_cpu_T,ligmpnn_cpu_id,ligmpnn_cpu_seq_rec,ligmpnn_cpu_overall_confidence,ligmpnn_cpu_ligand_confidence,ligmpnn_cpu_location
0,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0001,structure_1,51916.0,structure_1_0001,PLAEALRIILRGLALMARYGLALVRAAIAATRALLKRVAEDPVLLA...,0.1,1.0,0.485,0.4173,0.4173,/home/tripp/ProtFlow/examples/runners_example/...
1,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0002,structure_1,51916.0,structure_1_0002,ALEEAKEIIKKGLEFMKKYGEKIMKILLEATDALLKRIAKDPVLLA...,0.1,2.0,0.595,0.4302,0.4302,/home/tripp/ProtFlow/examples/runners_example/...
2,data/input_pdbs/structure_2.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_2_0001,structure_2,17511.0,structure_2_0001,ASPEELAAAAAAAALAAELAEGRAFLKAELPLFTAELLRAAGREDL...,0.1,1.0,0.420,0.4148,0.4148,/home/tripp/ProtFlow/examples/runners_example/...
3,data/input_pdbs/structure_2.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_2_0002,structure_2,17511.0,structure_2_0002,MSPEELEELEEEEEREERLREGRELLKRLLPLFTEALLRAAGKEEL...,0.1,2.0,0.510,0.3984,0.3984,/home/tripp/ProtFlow/examples/runners_example/...
4,data/input_pdbs/structure_3.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_3_0001,structure_3,44479.0,structure_3_0001,SAELRALLVELARLAGDPDPAAQDAFLGLAIALLGALALAALDPSL...,0.1,1.0,0.570,0.4577,0.4577,/home/tripp/ProtFlow/examples/runners_example/...
5,data/input_pdbs/structure_3.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_3_0002,structure_3,44479.0,structure_3_0002,MERLFALLVELARLAGDPDPEAQRRCIGLAIALLGALALAALDPAL...,0.1,2.0,0.605,0.4491,0.4491,/home/tripp/ProtFlow/examples/runners_example/...


In [8]:
my_poses.reset_poses(force_reset_df=True)
display(my_poses.df)

gpu_ligandmpnn = LigandMPNN(jobstarter=slurm_gpu_jobstarter)
my_poses = gpu_ligandmpnn.run(poses=my_poses, prefix='ligmpnn_gpu', nseq=2, model_type='protein_mpnn', jobstarter=slurm_gpu_jobstarter)
display(my_poses.df)

,input_poses,poses,poses_description
0,data/input_pdbs/structure_1.pdb,data/input_pdbs/structure_1.pdb,structure_1
1,data/input_pdbs/structure_2.pdb,data/input_pdbs/structure_2.pdb,structure_2
2,data/input_pdbs/structure_3.pdb,data/input_pdbs/structure_3.pdb,structure_3


sbatch: defined options
sbatch: -------------------- --------------------
sbatch: array               : 1-3%10
sbatch: cpus-per-task       : 2
sbatch: error               : /home/tripp/ProtFlow/examples/runners_example/ligmpnn_gpu//ligandmpnn_4799955_slurm.err
sbatch: gpus-per-node       : 1
sbatch: job-name            : ligandmpnn_4799955
sbatch: output              : /home/tripp/ProtFlow/examples/runners_example/ligmpnn_gpu//ligandmpnn_4799955_slurm.out
sbatch: verbose             : 3
sbatch: wrap                : eval `sed -n ${SLURM_ARRAY_TASK_ID}p /home/tripp/ProtFlow/examples/runners_example/ligmpnn_gpu//ligandmpnn_4799955_cmds`
sbatch: -------------------- --------------------
sbatch: end of defined options
sbatch: debug:  propagating RLIMIT_CPU=18446744073709551615
sbatch: debug:  propagating RLIMIT_FSIZE=18446744073709551615
sbatch: debug:  propagating RLIMIT_DATA=18446744073709551615
sbatch: debug:  propagating RLIMIT_STACK=67108864
sbatch: debug:  propagating RLIMIT_CORE=0
s

,input_poses,poses,poses_description,ligmpnn_gpu_mpnn_origin,ligmpnn_gpu_seed,ligmpnn_gpu_description,ligmpnn_gpu_sequence,ligmpnn_gpu_T,ligmpnn_gpu_id,ligmpnn_gpu_seq_rec,ligmpnn_gpu_overall_confidence,ligmpnn_gpu_ligand_confidence,ligmpnn_gpu_location
0,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0001,structure_1,73678.0,structure_1_0001,PLAEALRIIVRGLAYMAAHGEAHMARLLEATTALLRRIAQDPVLLA...,0.1,1.0,0.515,0.4308,0.4308,/home/tripp/ProtFlow/examples/runners_example/...
1,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0002,structure_1,73678.0,structure_1_0002,PLARALEIIKRGLELMKEKGEEFMKRLVEEMDELLARIAQDPVLLA...,0.1,2.0,0.505,0.4236,0.4236,/home/tripp/ProtFlow/examples/runners_example/...
2,data/input_pdbs/structure_2.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_2_0001,structure_2,19083.0,structure_2_0001,MSPEELEAERERAKLAAELEEARARLRSELPLFTAALLTAAGREEL...,0.1,1.0,0.470,0.3846,0.3846,/home/tripp/ProtFlow/examples/runners_example/...
3,data/input_pdbs/structure_2.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_2_0002,structure_2,19083.0,structure_2_0002,MSPEEIEAKKKEEERKKKLEEGRAFLKARLPEFAAELLRAAGKEEL...,0.1,2.0,0.545,0.3897,0.3897,/home/tripp/ProtFlow/examples/runners_example/...
4,data/input_pdbs/structure_3.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_3_0001,structure_3,38535.0,structure_3_0001,SAELRALLVELAKLAGDPDPAAQDRALGLAIALLAALALAALDPAR...,0.1,1.0,0.595,0.4554,0.4554,/home/tripp/ProtFlow/examples/runners_example/...
5,data/input_pdbs/structure_3.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_3_0002,structure_3,38535.0,structure_3_0002,MERLRALLVRLARLAGDPDPAAQDEALGLAIADLAARALAALRPEL...,0.1,2.0,0.625,0.4514,0.4514,/home/tripp/ProtFlow/examples/runners_example/...


You can provide options to your runner. These will be applied to all input poses. If you want to apply pose-specific options, you can use pose_options. The pose_options option reads in either a dataframe column (where each row contains the option for the corresponding pose) or a list of options for each pose. You can combine options and pose_options freely. If setting an option using both the generic options and pose_options, the pose_options will overwrite the generic one (but this should in general be avoided to prevent confusion). Some runners have explicit arguments for options that are mandatory or used very often (like model_type for LigandMPNN).

The options must be formatted in the same way as when running them directly. E.g. LigandMPNN uses options and flags with a -- prefix, whereas Rosetta uses options and flags indicated by -.

We are going to set general options to slightly bias ProteinMPNN to prefer Ala residues and provide a seed (to generate reproducible data, not recommended for general runs) and provide pose_options to preserve certain residues which should not be designed.

In [9]:
my_poses.reset_poses(force_reset_df=True)

pose_options = ["--fixed_residues 'A23 A69 AA126 A177'", "--fixed_residues 'A25 A73 A123 A177'", "--fixed_residues 'A26 A82 A119 A177'"]
my_poses.df['mpnn_pose_options'] = pose_options
print('Input poses with pose_options:')
display(my_poses.df[['poses', 'mpnn_pose_options']])

my_poses = ligandmpnn.run(poses=my_poses, prefix='options', nseq=10, model_type='protein_mpnn', options='--bias_AA A:0.1 --seed 111', pose_options='mpnn_pose_options')
print('Output poses:')
display(my_poses.df)

Input poses with pose_options:


,poses,mpnn_pose_options
0,data/input_pdbs/structure_1.pdb,--fixed_residues 'A23 A69 AA126 A177'
1,data/input_pdbs/structure_2.pdb,--fixed_residues 'A25 A73 A123 A177'
2,data/input_pdbs/structure_3.pdb,--fixed_residues 'A26 A82 A119 A177'


Output poses:


,input_poses,poses,poses_description,mpnn_pose_options,options_mpnn_origin,options_seed,options_description,options_sequence,options_T,options_id,options_seq_rec,options_overall_confidence,options_ligand_confidence,options_location
0,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0001,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0001,ALARAAAVIAAGLAAMAAHGREYWARELEATVELLKRISKDPVLLA...,0.1,1.0,0.5584,0.4258,0.4258,/home/tripp/ProtFlow/examples/runners_example/...
1,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0002,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0002,ALAAALAIIARGLAFYAAHGAAYHARELEATKALLARVAKDPVLLA...,0.1,2.0,0.5178,0.4157,0.4157,/home/tripp/ProtFlow/examples/runners_example/...
2,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0003,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0003,PLEKALEIIKKGLEYMKKYGEEYLKKEIEALKELLARIAKDPVLLA...,0.1,3.0,0.5533,0.4440,0.4440,/home/tripp/ProtFlow/examples/runners_example/...
3,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0004,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0004,PLAEALKIIAKGIAYMIAHGEEYFEEELAALKELLARIAKDPVLLA...,0.1,4.0,0.5533,0.4262,0.4262,/home/tripp/ProtFlow/examples/runners_example/...
4,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0005,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0005,ALARAAAIIARGLAAMAAHGAAYAAAAIADTRALLARIAQDPVLLA...,0.1,5.0,0.4822,0.4283,0.4283,/home/tripp/ProtFlow/examples/runners_example/...
5,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0006,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0006,PLAEALAIIARGLAFMAKHGEEYMKKELAATRALLARVAKDPVLLA...,0.1,6.0,0.5685,0.4233,0.4233,/home/tripp/ProtFlow/examples/runners_example/...
6,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0007,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0007,ALARALAIIARGVARMLAHGEEYWEKALAATKELLARIAKDPVLLK...,0.1,7.0,0.5330,0.4094,0.4094,/home/tripp/ProtFlow/examples/runners_example/...
7,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0008,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0008,ALERAKEIIKKGLEYMKEHGEEYMKKEIAATKELLARIAKDPVLLA...,0.1,8.0,0.5787,0.4348,0.4348,/home/tripp/ProtFlow/examples/runners_example/...
8,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0009,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0009,PLARALAIIARGLAAMAANGEAYRKRALEATDELLRRIAQDPVLLA...,0.1,9.0,0.4924,0.4140,0.4140,/home/tripp/ProtFlow/examples/runners_example/...
9,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0010,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0010,ALEEALEIIKRGLEYMAEYGDEYWERELAATGALLRRVAEDPVLLA...,0.1,10.0,0.5482,0.4225,0.4225,/home/tripp/ProtFlow/examples/runners_example/...


We can now predict the sequences generated by LigandMPNN by running ESMFold on our poses.

In [10]:
from protflow.tools.esmfold import ESMFold
esmfold = ESMFold(jobstarter=slurm_gpu_jobstarter)
my_poses = esmfold.run(poses=my_poses, prefix='esm_pred')
display(my_poses.df)

sbatch: defined options
sbatch: -------------------- --------------------
sbatch: array               : 1-10%10
sbatch: cpus-per-task       : 2
sbatch: error               : /home/tripp/ProtFlow/examples/runners_example/esm_pred//ESMFold_0000544_slurm.err
sbatch: gpus-per-node       : 1
sbatch: job-name            : ESMFold_0000544
sbatch: output              : /home/tripp/ProtFlow/examples/runners_example/esm_pred//ESMFold_0000544_slurm.out
sbatch: verbose             : 6
sbatch: wrap                : eval `sed -n ${SLURM_ARRAY_TASK_ID}p /home/tripp/ProtFlow/examples/runners_example/esm_pred//ESMFold_0000544_cmds`
sbatch: -------------------- --------------------
sbatch: end of defined options
sbatch: debug:  propagating RLIMIT_CPU=18446744073709551615
sbatch: debug:  propagating RLIMIT_FSIZE=18446744073709551615
sbatch: debug:  propagating RLIMIT_DATA=18446744073709551615
sbatch: debug:  propagating RLIMIT_STACK=67108864
sbatch: debug:  propagating RLIMIT_CORE=0
sbatch: debug:  propa

,input_poses,poses,poses_description,mpnn_pose_options,options_mpnn_origin,options_seed,options_description,options_sequence,options_T,options_id,options_seq_rec,options_overall_confidence,options_ligand_confidence,options_location,esm_pred_description,esm_pred_plddt,esm_pred_ptm,esm_pred_perresidue_plddt,esm_pred_location
0,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0001,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0001,ALARAAAVIAAGLAAMAAHGREYWARELEATVELLKRISKDPVLLA...,0.1,1.0,0.5584,0.4258,0.4258,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0001,49.590042,0.464904,"[29.43, 31.31, 31.39, 28.31, 33.59, 34.42, 32....",/home/tripp/ProtFlow/examples/runners_example/...
1,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0002,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0002,ALAAALAIIARGLAFYAAHGAAYHARELEATKALLARVAKDPVLLA...,0.1,2.0,0.5178,0.4157,0.4157,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0002,85.398094,0.907036,"[82.01, 82.84, 89.65, 90.52, 92.3, 88.19, 92.4...",/home/tripp/ProtFlow/examples/runners_example/...
2,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0003,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0003,PLEKALEIIKKGLEYMKKYGEEYLKKEIEALKELLARIAKDPVLLA...,0.1,3.0,0.5533,0.4440,0.4440,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0003,71.211914,0.723811,"[70.96, 80.75, 76.84, 74.47, 81.8, 82.94, 75.7...",/home/tripp/ProtFlow/examples/runners_example/...
3,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0004,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0004,PLAEALKIIAKGIAYMIAHGEEYFEEELAALKELLARIAKDPVLLA...,0.1,4.0,0.5533,0.4262,0.4262,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0004,50.365704,0.472184,"[33.51, 38.62, 40.52, 43.29, 38.52, 37.86, 41....",/home/tripp/ProtFlow/examples/runners_example/...
4,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0005,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0005,ALARAAAIIARGLAAMAAHGAAYAAAAIADTRALLARIAQDPVLLA...,0.1,5.0,0.4822,0.4283,0.4283,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0005,62.537060,0.611012,"[66.5, 72.18, 77.51, 63.63, 74.82, 80.08, 72.7...",/home/tripp/ProtFlow/examples/runners_example/...
5,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0006,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0006,PLAEALAIIARGLAFMAKHGEEYMKKELAATRALLARVAKDPVLLA...,0.1,6.0,0.5685,0.4233,0.4233,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0006,88.350235,0.930299,"[83.41, 87.19, 93.15, 86.59, 95.5, 90.51, 94.8...",/home/tripp/ProtFlow/examples/runners_example/...
6,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0007,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0007,ALARALAIIARGVARMLAHGEEYWEKALAATKELLARIAKDPVLLK...,0.1,7.0,0.5330,0.4094,0.4094,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0007,86.654144,0.916962,"[83.22, 82.74, 89.96, 77.33, 92.19, 87.45, 92....",/home/tripp/ProtFlow/examples/runners_example/...
7,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0008,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0008,ALERAKEIIKKGLEYMKEHGEEYMKKEIAATKELLARIAKDPVLLA...,0.1,8.0,0.5787,0.4348,0.4348,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0008,41.346474,0.268127,"[34.72, 37.19, 42.26, 37.75, 35.64, 40.5, 45.1...",/home/tripp/ProtFlow/examples/runners_example/...
8,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0009,--fixed_residues 'A23 A69 AA126 A177',structur

We can now calculate the backbone RMSD of our poses with the new sequences compared to the original input using rmsd metrics. Metrics work in the same way as runners, but just calculate scores and (in general) do not manipulate the poses.

In [11]:
import protflow.metrics.rmsd as rmsd_metrics
bb_rmsd_runner = rmsd_metrics.BackboneRMSD(ref_col='input_poses', atoms=['CA'], jobstarter=local_jobstarter)
my_poses = bb_rmsd_runner.run(poses=my_poses, prefix='CA_rmsd', overwrite=False)
display(my_poses.df)

,input_poses,poses,poses_description,mpnn_pose_options,options_mpnn_origin,options_seed,options_description,options_sequence,options_T,options_id,...,options_location,esm_pred_description,esm_pred_plddt,esm_pred_ptm,esm_pred_perresidue_plddt,esm_pred_location,CA_rmsd_index,CA_rmsd_description,CA_rmsd_location,CA_rmsd_rmsd
0,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0001,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0001,ALARAAAVIAAGLAAMAAHGREYWARELEATVELLKRISKDPVLLA...,0.1,1.0,...,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0001,49.590042,0.464904,"[29.43, 31.31, 31.39, 28.31, 33.59, 34.42, 32....",/home/tripp/ProtFlow/examples/runners_example/...,0,structure_1_0001,/home/tripp/ProtFlow/examples/runners_example/...,16.822071
1,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0002,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0002,ALAAALAIIARGLAFYAAHGAAYHARELEATKALLARVAKDPVLLA...,0.1,2.0,...,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0002,85.398094,0.907036,"[82.01, 82.84, 89.65, 90.52, 92.3, 88.19, 92.4...",/home/tripp/ProtFlow/examples/runners_example/...,1,structure_1_0002,/home/tripp/ProtFlow/examples/runners_example/...,0.617649
2,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0003,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0003,PLEKALEIIKKGLEYMKKYGEEYLKKEIEALKELLARIAKDPVLLA...,0.1,3.0,...,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0003,71.211914,0.723811,"[70.96, 80.75, 76.84, 74.47, 81.8, 82.94, 75.7...",/home/tripp/ProtFlow/examples/runners_example/...,2,structure_1_0003,/home/tripp/ProtFlow/examples/runners_example/...,10.307152
3,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0004,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0004,PLAEALKIIAKGIAYMIAHGEEYFEEELAALKELLARIAKDPVLLA...,0.1,4.0,...,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0004,50.365704,0.472184,"[33.51, 38.62, 40.52, 43.29, 38.52, 37.86, 41....",/home/tripp/ProtFlow/examples/runners_example/...,3,structure_1_0004,/home/tripp/ProtFlow/examples/runners_example/...,17.197902
4,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0005,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0005,ALARAAAIIARGLAAMAAHGAAYAAAAIADTRALLARIAQDPVLLA...,0.1,5.0,...,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0005,62.537060,0.611012,"[66.5, 72.18, 77.51, 63.63, 74.82, 80.08, 72.7...",/home/tripp/ProtFlow/examples/runners_example/...,4,structure_1_0005,/home/tripp/ProtFlow/examples/runners_example/...,10.989675
5,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0006,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0006,PLAEALAIIARGLAFMAKHGEEYMKKELAATRALLARVAKDPVLLA...,0.1,6.0,...,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0006,88.350235,0.930299,"[83.41, 87.19, 93.15, 86.59, 95.5, 90.51, 94.8...",/home/tripp/ProtFlow/examples/runners_example/...,5,structure_1_0006,/home/tripp/ProtFlow/examples/runners_example/...,0.450280
6,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0007,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,structure_1_0007,ALARALAIIARGVARMLAHGEEYWEKALAATKELLARIAKDPVLLK...,0.1,7.0,...,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0007,86.654144,0.916962,"[83.22, 82.74, 89.96, 77.33, 92.19, 87.45, 92....",/home/tripp/ProtFlow/examples/runners_example/...,6,structure_1_0007,/home/tripp/ProtFlow/examples/runners_example/...,0.670218
7,data/input_pdbs/structure_1.pdb,/home/tripp/ProtFlow/examples/runners_example/...,structure_1_0008,--fixed_residues 'A23 A69 AA126 A177',structure_1,111.0,s